In [47]:
from DrissionPage import WebPage
import json

In [48]:
driver = WebPage()

In [49]:
# Navigate to the page to establish session
driver.get("https://www.crunchbase.com/discover/saved/companies/414e9c05-7475-48d9-9082-9c0b293cfb79")

True

In [59]:
# Define the EXACT payload from the site
payload = {
    "field_ids": [
        "identifier",
        "categories",
        "location_identifiers",
        "short_description",
        "rank_org_company"
    ],
    "order": [
        {
            "field_id": "rank_org_company",
            "sort": "asc"
        }
    ],
    "query": [
        {
            "type": "predicate",
            "field_id": "last_funding_at",
            "operator_id": "eq",
            "values": ["2/22/25"]
        }
    ],
    "field_aggregators": [],
    "collection_id": "organization.companies",
    "limit": 15,
    "sort_by_new": True
}

print("Payload:")
print(json.dumps(payload, indent=2))

Payload:
{
  "field_ids": [
    "identifier",
    "categories",
    "location_identifiers",
    "short_description",
    "rank_org_company"
  ],
  "order": [
    {
      "field_id": "rank_org_company",
      "sort": "asc"
    }
  ],
  "query": [
    {
      "type": "predicate",
      "field_id": "last_funding_at",
      "operator_id": "eq",
      "values": [
        "2/22/25"
      ]
    }
  ],
  "field_aggregators": [],
  "collection_id": "organization.companies",
  "limit": 15,
  "sort_by_new": true
}


In [60]:
# Use run_js to make the fetch request - need to handle the Promise properly
js_code = f'''
const url = "https://www.crunchbase.com/v4/data/lists/organization.companies/414e9c05-7475-48d9-9082-9c0b293cfb79?source=list";
const payload = {json.dumps(payload)};

return fetch(url, {{
    method: "POST",
    headers: {{
        "Content-Type": "application/json",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.9",
        "Origin": "https://www.crunchbase.com",
        "Referer": "https://www.crunchbase.com/discover/saved/companies/414e9c05-7475-48d9-9082-9c0b293cfb79",
        "X-Requested-With": "XMLHttpRequest",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin"
    }},
    body: JSON.stringify(payload),
    credentials: "include"
}})
.then(response => response.json())
.then(data => data);
'''

result = driver.run_js(js_code)
print(f"\nResult type: {type(result)}")
print(f"Result is None: {result is None}")
if result:
    print(f"Count: {result.get('count', 0)} companies found")
    print(f"\nFirst 500 chars of response:")
    print(json.dumps(result, indent=2)[:500])
else:
    print("Result is None - trying alternative approach...")


Result type: <class 'dict'>
Result is None: False
Count: 11 companies found

First 500 chars of response:
{
  "count": 11,
  "entities": [
    {
      "uuid": "46264807-47be-4eb4-8ae7-2c6a3a722c62",
      "properties": {
        "identifier": {
          "permalink": "alpha-trade-ai",
          "image_id": "8d90db2fdc404d19bbcf8579692ed568",
          "uuid": "46264807-47be-4eb4-8ae7-2c6a3a722c62",
          "entity_def_id": "organization",
          "value": "Alpha Trade Ai"
        },
        "short_description": "Alpha Trade AI bridges the gap between retail and institutional traders with advance


In [58]:
print(result)

None


In [ ]:
# Extract and display the companies
if result and result.get('entities'):
    print(f"\n✓ Found {result.get('count', 0)} companies!\n")
    print("="*80)
    for i, entity in enumerate(result['entities'], 1):
        props = entity.get('properties', {})
        name = props.get('identifier', {}).get('value', 'N/A')
        desc = props.get('short_description', {}).get('value', 'N/A')
        categories = props.get('categories', {}).get('value', [])
        location = props.get('location_identifiers', {}).get('value', [])
        
        print(f"\n{i}. {name}")
        if categories:
            cat_names = [c.get('value', '') for c in categories if isinstance(c, dict)]
            print(f"   Categories: {', '.join(cat_names)}")
        if location:
            loc_names = [l.get('value', '') for l in location if isinstance(l, dict)]
            print(f"   Location: {', '.join(loc_names)}")
        if desc and desc != 'N/A':
            print(f"   Description: {desc[:150]}{'...' if len(desc) > 150 else ''}")
        print("-"*80)
else:
    print(f"\nResponse received but no entities found.")
    print(f"Count: {result.get('count', 0)}")
    print(f"\nRaw result keys: {list(result.keys()) if isinstance(result, dict) else 'Not a dict'}")

In [ ]:
# Save response to file
with open('crunchbase_response.json', 'w') as f:
    json.dump(result, f, indent=2)
    
print("\nResponse saved to crunchbase_response.json")